In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Stdlib
from collections import defaultdict, namedtuple
import math
import os
import tempfile

## Non-std libs
import matplotlib.pyplot as plt
import pandas as pd

## Local modules
from scn_rrd import rrd_meta_utils, rrd_utils

### Data rate consumed

In [ ]:
locations = [
    'FCS',
#     'lihi-southend', # Excluded b/c accounted for at OCC.
    'nickelsville-cd',
    'northlake',
    'occ-oromo',
    'sps-franklin',
    'sps-garfield',
]
portGroups = [
    'epc-backhaul-interface',
    'backhaul-interface',
]

meta = rrd_meta_utils.read_meta()
meta = meta[
    ( meta['location'].isin(locations) ) &
    ( meta['port_group_name'].isin(portGroups) )
]

In [ ]:
portId_to_consumeDf = {
    row['port_id'] :
    rrd_utils.read_rrd(row['hostname'], row['port_rrd_filename'], '-1month')
    for (_, row) in meta.iterrows()
}

In [ ]:
## Merge all ports for each device.
deviceHostname_to_consumeDf = dict()
for (_, row) in meta.iterrows():
    (deviceHostname, port_id) = row[[ 'hostname', 'port_id' ]]
    df = deviceHostname_to_consumeDf.get(deviceHostname, pd.DataFrame())
    port_df = portId_to_consumeDf[port_id]
    df = pd.concat([ df, port_df ], axis=0, ignore_index=True)
    deviceHostname_to_consumeDf[deviceHostname] = df

In [ ]:
axez_ct = len(deviceHostname_to_consumeDf)
fig, axez = plt.subplots(nrows=axez_ct, figsize=(12, 3 * axez_ct), constrained_layout=True)
for (ax, (deviceHostname, df)) in zip(axez, deviceHostname_to_consumeDf.items()):
    dt_sec = (df['time'][1] - df['time'][0]).seconds
    df_monthly = (
        df[['time', 'INOCTETS', 'OUTOCTETS']]
        .groupby(by=pd.Grouper(key='time', freq='D'))
        .sum()
        / pow(2, 30) * dt_sec
    )
    df_monthly.index = df_monthly.index.to_period('D')  # Format x ticks.
    df_monthly.plot(
        ax=ax,
        kind='bar',
        title=deviceHostname,
        xlabel='',
        ylabel='Gigabyte',
    )
fig.suptitle('Daily data usage')
;

### Data rate capacity (aka speedtest)

In [ ]:
MonitorDevice = namedtuple('MonitorDevice', ['location_label', 'login_env_key', 'consump_measure_hostname'])
monitor_devices = [
#     MonitorDevice('CD', 'CD_MON_LOGIN', 'CD-BackhaulGateway-hEXS'),
#     MonitorDevice('Franklin', 'FRANKLIN_MON_LOGIN', 'Franklin-EPC'),
#     MonitorDevice('Southend', 'SOUTHEND_MON_LOGIN', 'OCC-EPC'),
    # TODO above
    MonitorDevice('FCS', 'FCS_MON_LOGIN', 'FCS-EPC'),
]

In [ ]:
## Copy RRD files local to monitoring devices.
## In the future, we should deliver data from monitoring devices to LibreNMS, perhaps using SNMP Trap.

location_to_rrdName_to_capDf = defaultdict(dict)
for mondev in monitor_devices:
    login = rrd_meta_utils.DOTENV_ENTRIES[mondev.login_env_key]
    (addr, user, pw) = login.split('%%%%')
    
    for rrdName in ['down_rate', 'up_rate']:
        remote_rrd_filepath = f"~/speedtest/rrd/{rrdName}.rrd"
        
        with tempfile.NamedTemporaryFile() as f:
            scp_cmd = f"sshpass -p '{pw}' scp {user}@{addr}:{remote_rrd_filepath} {f.name}"
            ret = os.system(scp_cmd)
            assert ret == 0
            
            df = rrd_utils.rrd_to_dataframe(f.name, '-1month')

        location_to_rrdName_to_capDf[mondev.location_label][rrdName] = df

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))
for mondev in monitor_devices:
    down = location_to_rrdName_to_capDf[mondev.location_label]['down_rate']
    ax.plot(down['time'], down['down_rate'], label='inbound')
    
    up = location_to_rrdName_to_capDf[mondev.location_label]['up_rate']
    ax.plot(up['time'], up['up_rate'], label='outbound')
ax.legend()
fig.suptitle('Capacity');

In [ ]:
def utilization(consume_df: pd.DataFrame, capacity_df: pd.DataFrame, factor: float) -> pd.DataFrame:
    '''
    @arg consume_df and capacity_df:
        Each DF should have two columns,
        1st column containing time, and
        2nd column containing quantity (float).
    @return:
        1st column contains time, unmodified from @arg consume_df.
        2nd column contains (consumed quantity / capacity quantity),
            where capacity quantity comes from the last measurement at or before the time.
    '''
    cap_i = 0
    cap_n = len(capacity_df)
    
    perc_rows = []
    
    for (_, (consu_time, consu_quant)) in consume_df.iterrows():
        if pd.isna(consu_quant):
            continue
        
        while cap_i < cap_n and capacity_df.iloc[cap_i][0] <= consu_time:
            cap_i += 1

        if cap_i == 0:
            # No capacity was recorded before the current consumption timestamp.
            continue

        (cap_time, cap_quant) = capacity_df.iloc[cap_i-1]
        if pd.isna(cap_quant):
            # No capacity was recorded just before the current consumption timestamp.
            continue

        perc = consu_quant / cap_quant * factor
        perc_row = (consu_time, perc)
        perc_rows.append(perc_row)

    return pd.DataFrame(perc_rows, columns=['time', 'utilization'])

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))
for mondev in monitor_devices:
    consume_bytesPerSec_df = deviceHostname_to_consumeDf[mondev.consump_measure_hostname][['time', 'INOCTETS']]
    cap_MegabitsPerSec_df = location_to_rrdName_to_capDf[mondev.location_label]['down_rate']
    utilzn_df = utilization(consume_bytesPerSec_df, cap_MegabitsPerSec_df, 1/8 / pow(2, 20))
    ax.plot(utilzn_df['time'], utilzn_df['utilization'], label='inbound')
    
    consume_bytesPerSec_df = deviceHostname_to_consumeDf[mondev.consump_measure_hostname][['time', 'OUTOCTETS']]
    cap_MegabitsPerSec_df = location_to_rrdName_to_capDf[mondev.location_label]['up_rate']
    utilzn_df = utilization(consume_bytesPerSec_df, cap_MegabitsPerSec_df, 1/8 / pow(2, 20))
    ax.plot(utilzn_df['time'], utilzn_df['utilization'], label='outbound')
ax.legend()
fig.suptitle('Utilization');